In [4]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
import cv2
from torchvision import transforms
model.eval()

Using cache found in /home/zakaria/.cache/torch/hub/pytorch_vision_v0.10.0
/home/zakaria/miniconda3/envs/photoen/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zakaria/miniconda3/envs/photoen/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
newmodel = torch.nn.Sequential(*(list(model.children())[:-1]))

In [ ]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [6]:
image = cv2.imread("sample_images/a0676-kme_609.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
input = cv2.resize(image, (64, 64)) / 255.0
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input).unsqueeze(0).type(torch.float32)
pred =newmodel(input_tensor)

In [7]:
pred.shape

torch.Size([1, 512, 1, 1])

In [ ]:
image = cv2.imread("sample_images/a0676-kme_609_c.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
input = cv2.resize(image, (64, 64)) / 255.0
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input).unsqueeze(0).type(torch.float32)
target =newmodel(input_tensor)

In [ ]:
# import os 
# os.chdir('envs')

In [1]:
from envs.env_dataloader import create_dataloaders
train,test = create_dataloaders()

Encoding testing data ...


  0%|          | 0/4 [00:00<?, ?it/s]

finished...
Encoding training data ...


  0%|          | 0/36 [00:00<?, ?it/s]

finished...


In [2]:
f=train.dataset.encoded_source

In [3]:
f.shape

torch.Size([4500, 512])

In [ ]:
import torch
torch.cat([torch.rand(32,3,64,64),torch.rand(32,3,64,64),torch.rand(1,3,64,64)]).shape